In [5]:
import pandas as pd

In [6]:
df = pd.read_excel("jazzwerkstatt.xlsx", names=['obj1','obj2','titel','datum','jahr','umfang','maße','bemerk1','bemerk2','sig'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df.head()

,obj1,obj2,titel,datum,jahr,umfang,maße,bemerk1,bemerk2,sig
0,Akzidenz,Broschüre,Veranstaltungskalender Jazzwerkstatt Herbst/Wi...,,2019,14 ungezählte Seiten,21,<NA>,<NA>,Jazz A 1
1,Akzidenz,Eintrittskarte,Eintrittskarte Freitag-Ticket 50 € für 56. jaz...,2019-06-21 00:00:00,2019,1,21 x 8,jazzwerkstatt Peitz 56 21.-23. Juni 2019,Auflistung der Interpreten mit Uhrzeit,Jazz B 1
2,Akzidenz,Broschüre,Veranstaltungskalender Jazzwerkstatt 2022/2,,2022,14 ungezählte Seiten,21,jazzwerkstatt Peitz 59 09.-11.September 2022,<NA>,Jazz A 2
3,Akzidenz,Broschüre,Veranstaltungskalender Jazzwerkstatt II/2018,,2018,18 ungezählte Seiten,21,<NA>,<NA>,Jazz A 2
4,Akzidenz,Broschüre,Veranstaltungskalender Jazzwerkstatt I/2019,,2019,18 ungezählte Seiten,21,<NA>,<NA>,Jazz B 2


In [13]:
def pica_schreiben(row):

    pica =f"0600 yy\n"
#Satzart ermittelt mithilfe der Objektgattung
    if (row.obj1 == 'Akzidenz') and (row.obj2 == 'Broschüre'):
        pica += f"0500 Aa\n0501 Text$btxt\n0502 ohne Hilfsmittel zu benutzen$bn\n0503 Band$bnc\n1130 TB-Papier\n1132 a1-analog;a2-druck;f1-text;f2-blatt;f2-kodex;v-ganz\n"

    if (row.obj1 == 'Akzidenz') and (row.obj2 == 'Eintrittskarte'):
        pica += f"0500 Aa\n0501 Text$btxt\n0502 ohne Hilfsmittel zu benutzen$bn\n0503 Blatt$bnb\n1130 TB-Papier\n1132 a1-analog;a2-druck;f1-text;f2-blatt;f2-kodex;v-ganz\n"    

    if (row.obj1 == 'Akzidenz') and (row.obj2 == 'Handzettel'):
        pica += f"0500 Aa\n0501 Text$btxt\n0502 ohne Hilfsmittel zu benutzen$bn\n0503 Blatt$bnb\n1130 TB-Papier\n1132 a1-analog;a2-druck;f1-text;f2-blatt;f2-kodex;v-ganz\n"       

    if (row.obj1 == 'Plakat'):
        pica += f"0500 Pa\n0501 unbewegtes Bild$bsti\n0502 ohne Hilfsmittel zu benutzen$bn\n0503 Blatt$bnb\n1130 TB-Papier\n1132 a1-analog;f1-bild;v-ganz\n"
# 1100 Entweder rein das Jahr oder Jahr$nDatum
    if pd.notna(row.datum):
        pica += f"1100 {row.jahr}$n{row.datum}\n"
    else:
        pica += f"1100 {row.jahr}\n"

    pica +=f"1500 /1ger\n"

# 1700 wird anhand der Jahresangabe vergeben
    if 'year' in locals():
        if int(year[0]) > 1991:
            pica +=f"1700 /1XA-DE\n"
        elif int(year[0]) < 1990:
            pica +=f"1700 /1XA-DDDE\n"

    pica +=f"2105 94,P01\n"


    pica += f"""3100 !IDN Jazzwerkstatt!$BHerausgeber$4isb
4000 {row.titel} / Jazzwerkstatt Peitz
4046 Berlin
4060 {row.umfang}
"""
    if pd.notna(row.maße):
        pica +=f"4062 {row.maße} cm\n"

    pica +=f"4105 !IDN Qd-Datensatz!\n"


    if pd.notna(row.bemerk1):
        pica +=f"4201 {row.bemerk1}\n"

    if pd.notna(row.bemerk2):
        pica +=f"4201 {row.bemerk2}\n"

    #4256 link zum O*Datensatz
            
    pica +=f"4700 |BSM|\n"        

    if (row.obj1 == 'Plakat'):
        pica +=f"5590 [Objektgattung]\n5590 !04046198X!\n"
    if (row.obj1 == 'Akzidenz'):
        pica +=f"5590 [Objektgattung]\n5590 !041942175!\n"

    if (row.obj2 == 'Handzettel'):
        pica +=f"5591 !042930286!\n"
    if (row.obj2 == 'Eintrittskarte'):
        pica +=f"5591 !942218981!\n"        
    if (row.obj2 == 'Broschüre'):
        pica +=f"5591 !041467043!\n"

    pica +=f"""7002 x
4800 !009033645!
6710 !IDN Tq!
7100 {row.sig} @ m
7109 !!DBSM/S!! ; {row.sig}
"""
   
    
    pica +="\t\n"

    return pica

In [14]:
for index, row in df.iterrows():
    print(pica_schreiben(row))

0600 yy
0500 Aa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Band$bnc
1130 TB-Papier
1132 a1-analog;a2-druck;f1-text;f2-blatt;f2-kodex;v-ganz
1100 2019$n
1500 /1ger
2105 94,P01
3100 !IDN Jazzwerkstatt!$BHerausgeber$4isb
4000 Veranstaltungskalender Jazzwerkstatt Herbst/Winter 2019/20 / Jazzwerkstatt Peitz
4046 Berlin
4060 14 ungezählte Seiten
4062 21 cm
4105 !IDN Qd-Datensatz!
4700 |BSM|5590 [Objektgattung]
5590 !041942175!
5591 !041467043!
7002 x
4800 !009033645!
6710 !IDN Tq!
7100 Jazz A 1 @ m
7109 !!DBSM/S!! ; Jazz A 1
	

0600 yy
0500 Aa
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
1130 TB-Papier
1132 a1-analog;a2-druck;f1-text;f2-blatt;f2-kodex;v-ganz
1100 2019$n2019-06-21 00:00:00
1500 /1ger
2105 94,P01
3100 !IDN Jazzwerkstatt!$BHerausgeber$4isb
4000 Eintrittskarte Freitag-Ticket 50 € für 56. jazzwerkstatt Peitz / Jazzwerkstatt Peitz
4046 Berlin
4060 1
4062 21 x 8 cm
4105 !IDN Qd-Datensatz!
4201 jazzwerkstatt Peitz 56 21.-23. Juni 2019
4201 Auflist